In [ ]:
#Willem Master's Thesis

import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
plt.style.use ("bmh")
import warnings
import numpy as np
    
from sklearn_extra.cluster import KMedoids

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
# import both the operative network (n) as well as the optimized investment network n_optim

n = pypsa.Network(r"C:\Users\Willem\pypsa-eur\network_redispatch\elec_s_200_ec_lcopt_168H.nc")
n_optim = pypsa.Network(r"C:\Users\Willem\pypsa-eur\network_redispatch\solved\elec_s_200_ec_lcopt_168H.nc")

# Create operative network
def set_parameters_from_optimized(n, n_optim):
    '''
    Function to set optimized parameters from optimized network as nominal parameters of the operative network.
    -----
    Param:
    n: Optimized investment network.
    n_optim: Operative network for redispatch simulation.
    '''
    
    # set line capacities to optimized
    lines_typed_i = n.lines.index[n.lines.type != '']
    n.lines.loc[lines_typed_i, 'num_parallel'] = \
        n_optim.lines['num_parallel'].reindex(lines_typed_i, fill_value=0.)
    n.lines.loc[lines_typed_i, 's_nom'] = (
        np.sqrt(3) * n.lines['type'].map(n.line_types.i_nom) *
        n.lines.bus0.map(n.buses.v_nom) * n.lines.num_parallel)
    
    lines_untyped_i = n.lines.index[n.lines.type == '']
    for attr in ('s_nom', 'r', 'x'):
        n.lines.loc[lines_untyped_i, attr] = \
            n_optim.lines[attr].reindex(lines_untyped_i, fill_value=0.)
    n.lines['s_nom_extendable'] = False
    
    # set link capacities to optimized (HVDC links as well as store out/inflow links)
    links_dc_i = n.links.index[n.links.carrier == 'DC']
    n.links.loc[links_dc_i, 'p_nom'] = \
        n_optim.links['p_nom_opt'].reindex(links_dc_i, fill_value=0.)
    n.links.loc[links_dc_i, 'p_nom_extendable'] = False
    
    # set extendable generators to optimized and p_nom_extendable to False
    gen_extend_i = n.generators.index[n.generators.p_nom_extendable]
    n.generators.loc[gen_extend_i, 'p_nom'] = \
        n_optim.generators['p_nom_opt'].reindex(gen_extend_i, fill_value=0.)
    n.generators.loc[gen_extend_i, 'p_nom_extendable'] = False
    
    # set extendable storage unit power to ooptimized
    stor_extend_i = n.storage_units.index[n.storage_units.p_nom_extendable]
    n.storage_units.loc[stor_extend_i, 'p_nom'] = \
        n_optim.storage_units['p_nom_opt'].reindex(stor_extend_i, fill_value=0.)
    n.storage_units.loc[stor_extend_i, 'p_nom_extendable'] = False

    return n


# Call set parameters function
n = set_parameters_from_optimized(n, n_optim)


In [ ]:
#display(n_optim.generators_t.p)
#display(n_optim.generators)
#display(n_optim.loads_t.p_set.sum(axis=1))

df = pd.read_csv(r"C:\Users\Willem\pypsa-eur\data\costs.csv", sep = ',')
display(df)
n.components.components

In [77]:
for c in n_optim.iterate_components(list(n.components.keys())[2:]):
    print("{} has {} entries".format(c.name,len(c.df)))
    
for c in n.iterate_components(list(n.components.keys())[2:]):
    print("{} has {} entries".format(c.name,len(c.df)))    

Bus has 200 entries
Carrier has 8 entries
Line has 289 entries
LineType has 31 entries
TransformerType has 14 entries
Link has 3 entries
Load has 192 entries
Generator has 544 entries
StorageUnit has 400 entries
Bus has 200 entries
Carrier has 8 entries
Line has 289 entries
LineType has 31 entries
TransformerType has 14 entries
Link has 3 entries
Load has 192 entries
Generator has 544 entries
StorageUnit has 400 entries


In [89]:
def print_lopf_insights(network):
    # generator power
    print(" \nGenerator active power per snapshot:")
    print(network.generators_t.p)

    # network line flows
    print(" \nLine active power per snapshot:")
    print(network.lines_t.p0)
    
    # relative line loading
    print(" \nRelative line loading per snapshot:")
    print(abs(network.lines_t.p0)/network.lines.s_nom)
    max_loading = (network.lines_t.p0/network.lines.s_nom).max(axis = 0)
    print(max_loading)

    #In linear approximation, all voltage magnitudes are nominal, i.e. 1 per unit
    print(" \nVoltage magnitude at nodes:")
    print(network.buses_t.v_mag_pu)

    #At bus 2 the price is set above any marginal generation costs in the model, because to dispatch to
    #it from expensive generator 0, also some dispatch from cheap generator 1 has to be substituted from generator0
    #to avoid overloading line 1.
    print(" \nMarginal prices at nodes:")
    print(network.buses_t.marginal_price)


def add_BESS_loadflexibility(network, network_year, c_rate = 0.25, flex_share = 0.1):
    '''
    Adds battery storages at every node, depending on the flexibility potential of the loads allocated to this node. 
    Methodology according to Network development plan and Frontier study:
        - Flexibility potential at load-nodes: Flexibility Potential through stationary battery storage in distribution grids
        - Flexibility potential at power plants: Flexibility through battery installations at powerplants (by operators)
    
    Following the assumption of x percentage of frontier flexibility potential being installed at node according to loads. 
    Flexibility potential can e.g. be aggregated powerwall capacity at the node.
    -----
    Param:
    network: network from dispatch optimization    
    '''
    # Clean up all previously saved battery components
    clean_batteries(network)
    
    # get bus names
    bus_names = network.buses.index.tolist()
    
    # get mean load at every bus
    df_loads = network_year.loads
    df_loads["p_mean"] = network_year.loads_t.p_set.mean(axis = 0)
    
    # According to x% rule, add energy stores with energy = x% of average load at bus
    for i in range(len(bus_names)):
        bus_name = network.buses.index.tolist()[i]
        battery_bus = "{}_{}".format(bus_name,"BESS")
        
        # determin flexibility at bus
        df_loads_bus = df_loads[df_loads["bus"] == bus_name]
        
        ################
        
        
        # TODO: ÄNDERN DER maximalen energie: Die speicherbare energie soll abhängig vom mittelwert der last 
        # über den GESAMTEN zeitraum sein, nicht nur vom mittelwert des aktuellen (1 day) networks!
        
        
        ################
        
        if not df_loads_bus.empty:
            p_flex = df_loads_bus.p_mean * flex_share
        else:
            p_flex = 0
            
        print("p_flex = " + str(p_flex))

        # add additional bus solely for representation of battery at location of previous bus
        network.add("Bus",
                    name = battery_bus,
                    x = network.buses.loc[bus_name,"x"],
                    y = network.buses.loc[bus_name,"y"],
                    carrier = "battery")
        # add store
        network.add("Store", name = "BESS_{}".format(i),
                    bus = battery_bus,
                    e_nom = p_flex / c_rate,
                    e_nom_extendable = False,
                    e_min_pu = 0,
                    e_max_pu = 1,
                    e_initial = 0.5,
                    e_cyclic = True,
                    p_set = p_flex,
                    q_set = 0.05,
                    marginal_cost = 0,
                    capital_cost = 0,
                    standing_loss = 0)
        # discharge link
        network.add("Link",
                    name = "BESS_{}_discharger".format(i+1),
                    bus0 = battery_bus,
                    bus1 = bus_name,
                    capital_cost = 0,
                    p_nom = p_flex,
                    p_nom_extendable = False,
                    p_max_pu = 1,
                    p_min_pu = 0,
                    marginal_cost = 0,
                    efficiency = 0.96)
        # charge link
        network.add("Link",
                    name = "BESS_{}_charger".format(i+1),
                    bus0 = bus_name,
                    bus1 = battery_bus,
                    capital_cost = 0,
                    p_nom = p_flex,
                    p_nom_extendable = False,
                    p_max_pu = 1,
                    p_min_pu = 0,
                    marginal_cost = 0,
                    efficiency = 0.96)
    network.name = str(network.name) + " BESS loadflexibility"
        
def add_BESS(network):
    '''
    Adds static battery storages (fixed capacity) to the base network
    Removes previously added batteries (links and stores) from the network
    
    A battery is added to the network by combining a link for discharge, and one for charge (representing inverter operations)
    and a store unit representing the battery capacity
    '''
    # Clean up all previously saved battery components
    clean_batteries(network)
    
    bus_names = network.buses.index.tolist()
    for i in range(len(bus_names)):
        bus_name = bus_names[i]
        battery_bus = "{}_{}".format(bus_name,"BESS")
        # add additional bus solely for representation of battery at location of previous bus
        network.add("Bus",
                    name = battery_bus,
                    x = network.buses.loc[bus_name,"x"],
                    y = network.buses.loc[bus_name,"y"],
                    carrier = "battery")
        # discharge link
        network.add("Link",
                    name = "Battery_{}_dCH".format(i+1),
                    bus0 = battery_bus,
                    bus1 = bus_name,
                    capital_cost = 0,
                    p_nom = 150,
                    p_nom_extendable = False,
                    p_max_pu = 1,
                    p_min_pu = 0,
                    marginal_cost = 0,
                    efficiency = 0.96)
        # charge link
        network.add("Link",
                    name = "Battery_{}_CH".format(i+1),
                    bus0 = bus_name,
                    bus1 = battery_bus,
                    capital_cost = 0,
                    p_nom = 150,
                    p_nom_extendable = False,
                    p_max_pu = 1,
                    p_min_pu = 0,
                    marginal_cost = 0,
                    efficiency = 0.96)
        # add store
        network.add("Store", name = "BESS_{}".format(i),    
                    bus = battery_bus,
                    e_nom = 200,
                    e_nom_extendable = False,
                    e_min_pu = 0,
                    e_max_pu = 1,
                    e_initial = 0.5,
                    e_cyclic = True,
                    p_set = 100,
                    q_set = 0.05,
                    marginal_cost = 0,
                    capital_cost = 0,
                    standing_loss = 0)
    network.name = str(network.name) + " BESS"
    
        
def clean_generators(network):
    """
    Remove all generators from network
    """
    network.mremove("Generator", [name for name in network.generators.index.tolist()])


def build_redispatch_network(network, network_dispatch):
    '''
    Uses predefined component building functions for building the redispatch network.
    Passes the dispatch per generator as fixed generation for each generator.
    Adds a ramp down generator (neg. redispatch) and ramp up generator (pos. redispatch) for each conventional generator.
    Adds a ramp down generator (curtailment) for each EE generator.
    -----
    Parameters:
    network: network representing one day of snapshots (not optimized)
    network_dispatch: network from dispatch optimization (one node market model)
    '''
    # Copy initial network and remove generators for redispatch step
    network_redispatch = network.copy()
    clean_generators(network_redispatch)
    
    # Add new generators for redispatch calculation
    l_generators = network.generators.index.tolist()
    l_conv_carriers = ["hard coal","lignite","gas","oil","nuclear","OCGT","CCGT"]
    
    for generator in l_generators:
        
        # For each conventional generator in network: Add 3 generators in network_redispatch
        if network.generators.loc[generator]["carrier"] in l_conv_carriers:
            # Base generator from network, power range is fixed by dispatch simulation results (therefore runs at 0 cost)
            network_redispatch.add("Generator",
                                   name = "{}".format(generator), 
                                   bus = network.generators.loc[generator]["bus"],
                                   p_nom = network.generators.loc[generator]["p_nom"],
                                   efficiency = network.generators.loc[generator]["efficiency"],
                                   marginal_cost = 0,
                                   capital_cost = 0,
                                   p_max_pu = (network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                   p_min_pu = (network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                  )
            # Upwards generator (positive redispatch) ranges within the difference of the base power p_max_pu (e.g. 0.92) and p_max_pu = 1
            network_redispatch.add("Generator",
                                   name = "{}_pos".format(generator), 
                                   bus = network.generators.loc[generator]["bus"],
                                   p_nom = network.generators.loc[generator]["p_nom"],
                                   efficiency = network.generators.loc[generator]["efficiency"],
                                   marginal_cost = network.generators.loc[generator]["marginal_cost"],
                                   capital_cost = 0,
                                   p_max_pu = (1 - network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                   p_min_pu = 0,
                                  )
            # Downwards generator (negative redispatch): Using this generator reduces the energy feed in at specific bus, 
            # but increases costs for curtialment. Cost for curtailment = lost profit for not feeding in - savings through ramp down 
            network_redispatch.add("Generator",
                                   name = "{}_neg".format(generator), 
                                   bus = network.generators.loc[generator]["bus"],
                                   p_nom = network.generators.loc[generator]["p_nom"],
                                   efficiency = network.generators.loc[generator]["efficiency"],
                                   marginal_cost = (network.generators.loc[generator]["marginal_cost"] - network_dispatch.buses_t.marginal_price[network_dispatch.generators.loc[generator]["bus"]]).tolist(),
                                   capital_cost = 0,
                                   p_max_pu = 0,
                                   p_min_pu = (- network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                  )
            
        else:
            # For renewable sources: Only add base and negative generator, representing the curtailment of EE 
            network_redispatch.add("Generator",
                                   name = "{}".format(generator), 
                                   bus = network.generators.loc[generator]["bus"],
                                   p_nom = network.generators.loc[generator]["p_nom"],
                                   efficiency = network.generators.loc[generator]["efficiency"],
                                   marginal_cost = 0,
                                   capital_cost = 0,
                                   p_max_pu = (network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                   p_min_pu = (network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                  )
            # Downwards generator (negative redispatch): Using this generator reduces the energy feed in at specific bus, 
            # but increases costs for curtialment. Cost for curtailment = lost profit for not feeding in - savings through ramp down 
            network_redispatch.add("Generator",
                                   name = "{}_neg".format(generator), 
                                   bus = network.generators.loc[generator]["bus"],
                                   p_nom = network.generators.loc[generator]["p_nom"],
                                   efficiency = network.generators.loc[generator]["efficiency"],
                                   marginal_cost = (network.generators.loc[generator]["marginal_cost"] - network_dispatch.buses_t.marginal_price[network_dispatch.generators.loc[generator]["bus"]]).tolist(),
                                   capital_cost = 0,
                                   p_max_pu = 0,
                                   p_min_pu = (- network_dispatch.generators_t.p[generator] / network_dispatch.generators.loc[generator]["p_nom"]).tolist(),
                                  )
    
    return network_redispatch


def build_market_model(network):
    '''
    Clusters all generators, loads, stores and storage units to a single (medoid) bus to emulate market simulation.

    TODO: For multiple countries, the dataset to be clusterd first needs to be separated by country (groupby("country")) and then clustered.
    '''   
    network_dispatch = network.copy()
    
    # cluster all buses to the center one and assign all grid elements to that bus
    df_busmap = network_dispatch.buses

    # Cluster buses to 1 cluster & compute center
    np_busmap = df_busmap[["x","y"]].to_numpy()

    # Compute cluster score
    k_medoids = KMedoids(n_clusters=1, metric='euclidean', init='k-medoids++', max_iter=300, random_state=None)
    score = k_medoids.fit(np_busmap)

    x_medoid = k_medoids.cluster_centers_[0][0]
    y_medoid = k_medoids.cluster_centers_[0][1]

    # find single medoid bus in dataframe
    market_bus = df_busmap[(df_busmap["x"] == x_medoid) & (df_busmap["y"] == y_medoid)]
    bus_name = str(market_bus.index.values)[2:-2] # save bus name, remove brackets & quotemarks

    # assign all generators, loads, storage_units, stores to the medoid bus (market_bus)
    network_dispatch.loads["bus"] = bus_name
    network_dispatch.generators["bus"] = bus_name
    network_dispatch.stores["bus"] = bus_name
    network_dispatch.storage_units["bus"] = bus_name
    return network_dispatch


#def solve_dispatch_network(network)

def solve_redispatch_network(network, network_dispatch):
    '''
    Calls the building function and solves the network under redispatch constraints.
    -----
    Parameters:
    network: network from dispatch optimization
    '''
    #### TODO: add extra functionality that only pos OR negative can be =! 0 during 1 snapshot    

    network_redispatch = build_redispatch_network(network, network_dispatch)
    # Solve new network
    network_redispatch.lopf(solver_name="gurobi", 
                 formulation ="kirchhoff")
    
    #print_lopf_insights(network_redispatch)
    
    # Plot line loadings 
    series_loading  = ((network_redispatch.lines_t.p0.abs().mean().sort_index() / 
                        (network_redispatch.lines.s_nom_opt*network_redispatch.lines.s_max_pu)
                        .sort_index()).fillna(0.))

    fig,ax = plt.subplots(figsize = (10,10),subplot_kw = {"projection": ccrs.PlateCarree()})
    
    network_redispatch.plot(ax=ax,
           bus_colors ="gray",
           branch_components = ["Line"],
           line_widths = network_redispatch.lines.s_nom_opt/3e3,
           line_colors = series_loading,
           line_cmap=plt.cm.viridis,
           color_geomap = True,
           bus_sizes = 0,
           geomap = True)
    ax.axis("on")
    return network_redispatch
    
    
# Redispatch workflow with batteries
def build_redispatch_network_with_bat(network, network_dispatch, network_year, c_rate, flex_share, flex_store = True):
    '''
    Uses predefined component building functions for building the redispatch network and adds batteries.
    -----
    Parameters:
    network: network from dispatch optimization 
    '''
    network_redispatch_bat = build_redispatch_network(network, network_dispatch)
    # Adding a battery at every node
    if flex_store == True:
        add_BESS_loadflexibility(network_redispatch_bat, network_year, c_rate, flex_share)
    else:
        add_BESS(network_redispatch_bat)
        
    return network_redispatch_bat
    
def solve_redispatch_network_with_bat(network, network_dispatch, network_year, c_rate, flex_share,flex_store = True):
    '''
    Calls redispatch building function and solves the network.
    -----
    Parameters:
    network: network from dispatch optimization 
    '''
    
    #### TODO: add extra functionality that only pos OR negative can be =! 0 during 1 snapshot    

    network_redispatch_bat = build_redispatch_network_with_bat(network, network_dispatch, network_year, c_rate, flex_share,flex_store = True,)
    # Solve new network
    network_redispatch_bat.lopf(solver_name="gurobi", formulation ="kirchhoff")
    
    # Plot line loadings 
    series_loading  = ((network_redispatch_bat.lines_t.p0.abs().mean().sort_index() / (network_redispatch_bat.lines.s_nom_opt*network_redispatch_bat.lines.s_max_pu)
                    .sort_index()).fillna(0.))

    fig,ax = plt.subplots(figsize = (10,10),subplot_kw = {"projection": ccrs.PlateCarree()})
    
    network_redispatch_bat.plot(ax=ax,
           bus_colors ="gray",
           branch_components = ["Line"],
           line_widths = network_redispatch_bat.lines.s_nom_opt/3e3,
           line_colors = series_loading,
           line_cmap=plt.cm.viridis,
           color_geomap = True,
           bus_sizes = 0,
           geomap = True)
    ax.axis("on")
    return network_redispatch_bat



def concat_network(l_networks, scenario):
    '''
    Function that merges technically identical, but temporally decoupled networks by concatenating 
    their time-dependent components (input & output)
    -----
    Param:
    l_networks: list of daily solved networks (can be either l_networks_dispatch or l_networks_redispatch)
    scenario: Determines whether input networks will have stores or not
    '''

    #create new network out of first network of the list of identical networks for time indipendent components
    network_concat = l_networks[0].copy()

    # ---------------
    # For each network in the results list: Extract the respective dataframes of interest
    # ---------------
    
    # Concat snapshots first
    l_snapshots = []
    for network in l_networks:
        l_snapshots.append(network.snapshots)
    idx = l_snapshots[0]
    network_concat.snapshots = idx.union_many(l_snapshots)
    
    # time dependent components
    if scenario == "bat":
        network_components_t = ["buses_t","generators_t","lines_t","stores_t","storage_units_t"]
    else:
        network_components_t = ["buses_t","generators_t","lines_t","storage_units_t"]

    # append all time dependant network component data
    for network_component in network_components_t:
        for component in getattr(network_concat, network_component):

            # create a list for the daily networks data of that component
            l_component_t = []

            # append the components data from all daily network to their respective list
            for network in l_networks:
                # check component to be concated
                l_component_t.append(getattr(getattr(network, network_component), component))

            # set the time dependent attribute to the net network
            setattr(getattr(network_concat, network_component), component, pd.concat(l_component_t, axis = 0))
    
    # Concat objective value
    network_concat.objective = 0
    for network in l_networks:
        if hasattr(network, 'objective'):
            network_concat.objective = network_concat.objective + network.objective    
    
    return network_concat

In [67]:
len(n.snapshots)
df_snapshots = pd.DataFrame(n.snapshots)
print(len(df_snapshots.index))

53


In [1]:
def redispatch_workflow(network, network_optim, scenario = "no bat",  c_rate = 0.25, flex_share = 0.1 ,flex_store = True):
    '''
    Function for executing the whole redispatch workflow.
    
    -----
    Param:
    network: Un-optimized network from pypsa-eur.
    network_optim: Already optimized network from the strategic network optimization in pypsa-eur.
    scenario: "bat" or "no bat" decides whether the redispatch optimization is run with or without batteries.
    
    '''
    # create lists to save results
    network_year = network
    l_networks_24 = []
    l_networks_dispatch = []
    l_networks_redispatch = []
    
    # Generate operative pypsa-eur network without investment problem
    network = set_parameters_from_optimized(network, network_optim)
    
    # Only dispatch optimization: Capital costs set to zero
    network.generators.loc[:,"capital_cost"] = 0
    
    # Run dispatch_redispatch workflow for every day (24h batch)
    for start_hour in range(0, len(network.snapshots), 24):

        #print(start_hour)
        n_24 = network.copy(snapshots = network.snapshots[start_hour:start_hour+24])
        
        # ------------------
        # Build market model, solve dispatch network and plot network insights
        # ------------------
        n_dispatch = build_market_model(n_24)
        
        n_dispatch.lopf(solver_name="gurobi", formulation ="kirchhoff")
        #print_lopf_insights(n_dispatch)

        # print line loadings 
        series_loading  = ((n_dispatch.lines_t.p0.abs().mean().sort_index() / 
                            (n_dispatch.lines.s_nom_opt*n_dispatch.lines.s_max_pu)
                            .sort_index()).fillna(0.))

        fig,ax = plt.subplots(figsize = (10,10),subplot_kw = {"projection": ccrs.PlateCarree()})

        n_dispatch.plot(ax=ax,
                       bus_colors ="gray",
                       branch_components = ["Line"],
                       line_widths = n_dispatch.lines.s_nom_opt/3e3,
                       line_colors = series_loading,
                       line_cmap=plt.cm.viridis,
                       color_geomap = True,
                       bus_sizes = 0,
                       geomap = True)
        ax.axis("on")
        
        print("\n now\n RE-dispatch\n simulation\n")
        
        ## plot dispatch highlights
        #display(n_dispatch.generators_t.p.sum(axis=1))
        #display(n_dispatch.generators_t.p)
        #display(n_dispatch.loads_t.p_set.sum(axis = 1))
        #display(n_dispatch.buses_t.p)

        # ------------------
        # Call redispatch optimization and plot network insights
        # ------------------
        if scenario == "no bat":
            n_redispatch = solve_redispatch_network(n_24, n_dispatch)
        elif scenario == "bat":
            n_redispatch = solve_redispatch_network_with_bat(n_24, n_dispatch, network_year, c_rate = 0.25, flex_share = 0.1, flex_store = True)
        
        print("\n RE-dispatch\n simulation\n done\n")
        
        # ------------------
        # Append networks to corresponding yearly lists
        # ------------------
        l_networks_24.append(n_24)
        l_networks_dispatch.append(n_dispatch)
        l_networks_redispatch.append(n_redispatch)

    # ---------------
    # For each results list: Create a network out of the daily networks
    # ---------------
    network_control = concat_network(l_networks_24, scenario)
    network_dispatch = concat_network(l_networks_dispatch, scenario)
    network_redispatch = concat_network(l_networks_redispatch, scenario)

    l_results = [network_control, network_dispatch, network_redispatch]
        
    return l_results

    
    
# Run the redispatch workflow
l_networks_dispatch = redispatch_workflow(n, n_optim, scenario = "no bat")
    
    





NameError: name 'n' is not defined

In [58]:
display(n_dispatch.generators[["p_nom","p_max_pu"]])
display(n.generators[["p_nom","p_max_pu"]])


display(n_dispatch.generators_t.p)
display(n_dispatch.generators.p_nom)


display(n_redispatch.generators_t.p_max_pu)
display(n_dispatch.lines.s_nom_opt)


,p_nom,p_max_pu
name,,
DE0 0 OCGT,0.000000,1.0
DE0 0 onwind,0.000000,1.0
DE0 0 solar,0.000000,1.0
DE0 1 CCGT,823.000000,1.0
DE0 1 OCGT,0.000000,1.0
...,...,...
DE0 97 solar,1880.647060,1.0
DE0 98 onwind,0.000000,1.0
DE0 98 solar,106.635102,1.0


,p_nom,p_max_pu
name,,
DE0 0 OCGT,0.000000,1.0
DE0 0 onwind,0.000000,1.0
DE0 0 solar,0.000000,1.0
DE0 1 CCGT,823.000000,1.0
DE0 1 OCGT,0.000000,1.0
...,...,...
DE0 97 solar,1880.647060,1.0
DE0 98 onwind,0.000000,1.0
DE0 98 solar,106.635102,1.0


name,DE0 0 OCGT,DE0 0 onwind,DE0 0 solar,DE0 1 CCGT,DE0 1 OCGT,DE0 1 onwind,DE0 1 solar,DE0 10 OCGT,DE0 10 onwind,DE0 10 solar,DE0 100 CCGT,DE0 100 OCGT,DE0 100 onwind,DE0 100 solar,DE0 101 CCGT,DE0 101 OCGT,DE0 101 onwind,DE0 101 solar,DE0 102 CCGT,DE0 102 OCGT,DE0 102 onwind,DE0 102 solar,DE0 103 onwind,DE0 103 solar,DE0 104 onwind,DE0 104 solar,DE0 105 onwind,DE0 105 solar,DE0 106 OCGT,DE0 106 onwind,DE0 106 solar,DE0 107 onwind,DE0 107 solar,DE0 108 onwind,DE0 108 solar,DE0 109 onwind,DE0 109 solar,DE0 11 OCGT,DE0 11 onwind,DE0 11 solar,DE0 110 CCGT,DE0 110 OCGT,DE0 110 onwind,DE0 110 solar,DE0 111 onwind,DE0 111 solar,DE0 112 OCGT,DE0 112 onwind,DE0 112 solar,DE0 113 onwind,DE0 113 solar,DE0 114 onwind,DE0 114 solar,DE0 115 onwind,DE0 115 solar,DE0 117 onwind,DE0 117 solar,DE0 118 onwind,DE0 118 solar,DE0 119 OCGT,DE0 119 onwind,DE0 119 solar,DE0 12 CCGT,DE0 12 OCGT,DE0 12 onwind,DE0 12 solar,DE0 120 OCGT,DE0 120 onwind,DE0 120 solar,DE0 121 onwind,DE0 121 solar,DE0 122 OCGT,DE0 122 offwind-ac,DE0 122 offwind-dc,DE0 122 onwind,DE0 122 solar,DE0 123 OCGT,DE0 123 onwind,DE0 123 solar,DE0 124 onwind,DE0 124 solar,DE0 125 onwind,DE0 125 solar,DE0 126 OCGT,DE0 126 onwind,DE0 126 solar,DE0 127 OCGT,DE0 127 onwind,DE0 127 solar,DE0 128 onwind,DE0 128 solar,DE0 129 onwind,DE0 129 solar,DE0 13 OCGT,DE0 13 onwind,DE0 13 solar,DE0 130 onwind,DE0 130 solar,DE0 131 onwind,DE0 131 solar,DE0 132 OCGT,DE0 132 onwind,DE0 132 solar,DE0 133 OCGT,DE0 133 onwind,DE0 133 solar,DE0 134 onwind,DE0 134 solar,DE0 135 onwind,DE0 135 solar,DE0 136 onwind,DE0 136 solar,DE0 137 CCGT,DE0 137 OCGT,DE0 137 onwind,DE0 137 solar,DE0 138 onwind,DE0 138 solar,DE0 139 OCGT,DE0 139 onwind,DE0 139 solar,DE0 14 CCGT,DE0 14 OCGT,DE0 14 onwind,DE0 14 solar,DE0 140 OCGT,DE0 140 onwind,DE0 140 solar,DE0 141 OCGT,DE0 141 onwind,DE0 141 solar,DE0 142 onwind,DE0 142 solar,DE0 143 onwind,DE0 143 solar,DE0 144 onwind,DE0 144 solar,DE0 145 onwind,DE0 145 solar,DE0 146 onwind,DE0 146 solar,DE0 147 onwind,DE0 147 solar,DE0 148 CCGT,DE0 148 OCGT,DE0 148 onwind,DE0 148 solar,DE0 149 onwind,DE0 149 solar,DE0 15 CCGT,DE0 15 OCGT,DE0 15 onwind,DE0 15 solar,DE0 151 offwind-ac,DE0 151 offwind-dc,DE0 151 onwind,DE0 151 solar,DE0 152 onwind,DE0 152 solar,DE0 153 onwind,DE0 153 solar,DE0 154 CCGT,DE0 154 OCGT,DE0 154 onwind,DE0 154 solar,DE0 155 OCGT,DE0 155 onwind,DE0 155 solar,DE0 156 onwind,DE0 156 solar,DE0 158 onwind,DE0 158 solar,DE0 159 CCGT,DE0 159 OCGT,DE0 159 onwind,DE0 159 solar,DE0 16 CCGT,DE0 16 OCGT,DE0 16 offwind-ac,DE0 16 onwind,DE0 16 solar,DE0 161 onwind,DE0 161 solar,DE0 162 OCGT,DE0 162 onwind,DE0 162 solar,DE0 163 onwind,DE0 163 solar,DE0 164 onwind,DE0 164 solar,DE0 165 OCGT,DE0 165 onwind,DE0 165 solar,DE0 166 OCGT,DE0 166 onwind,DE0 166 solar,DE0 167 onwind,DE0 167 solar,DE0 168 onwind,DE0 168 solar,DE0 169 CCGT,DE0 169 OCGT,DE0 169 onwind,DE0 169 solar,DE0 17 CCGT,DE0 17 OCGT,DE0 17 onwind,DE0 17 solar,DE0 170 CCGT,DE0 170 OCGT,DE0 170 onwind,DE0 170 solar,DE0 171 OCGT,DE0 171 onwind,DE0 171 solar,DE0 173 onwind,DE0 173 solar,DE0 174 onwind,DE0 174 solar,DE0 175 onwind,DE0 175 solar,DE0 176 onwind,DE0 176 solar,DE0 177 OCGT,DE0 177 onwind,DE0 177 solar,DE0 178 onwind,DE0 178 solar,DE0 179 onwind,DE0 179 solar,DE0 18 onwind,DE0 18 solar,DE0 180 onwind,DE0 180 solar,DE0 181 onwind,DE0 181 solar,DE0 183 onwind,DE0 183 solar,DE0 184 onwind,DE0 184 solar,DE0 185 onwind,DE0 185 solar,DE0 186 offwind-ac,DE0 186 offwind-dc,DE0 186 onwind,DE0 186 solar,DE0 187 OCGT,DE0 187 onwind,DE0 187 solar,DE0 188 OCGT,DE0 188 onwind,DE0 188 solar,DE0 189 OCGT,DE0 189 onwind,DE0 189 solar,DE0 19 CCGT,DE0 19 OCGT,DE0 19 onwind,DE0 19 solar,DE0 190 OCGT,DE0 190 onwind,DE0 190 solar,DE0 191 OCGT,DE0 191 onwind,DE0 191 solar,DE0 192 onwind,DE0 192 solar,DE0 193 onwind,DE0 193 solar,DE0 194 onwind,DE0 194 solar,DE0 195 OCGT,DE0 195 onwind,DE0 195 solar,DE0 196 onwind,DE0 196 solar,DE0 197 offwind-ac,DE0 197 onwind,DE0 197 solar,DE0 198 onwind,DE0 198 solar,DE0 2 CCGT,DE0 2 OCGT,DE0 2 o

name
DE0 0 OCGT          0.000000
DE0 0 onwind        0.000000
DE0 0 solar         0.000000
DE0 1 CCGT        823.000000
DE0 1 OCGT          0.000000
                    ...     
DE0 97 solar     1880.647060
DE0 98 onwind       0.000000
DE0 98 solar      106.635102
DE0 99 onwind       0.000000
DE0 99 solar        0.000000
Name: p_nom, Length: 544, dtype: float64

,DE0 0 OCGT,DE0 0 OCGT_pos,DE0 0 onwind,DE0 0 solar,DE0 1 CCGT,DE0 1 CCGT_pos,DE0 1 OCGT,DE0 1 OCGT_pos,DE0 1 onwind,DE0 1 solar,DE0 10 OCGT,DE0 10 OCGT_pos,DE0 10 onwind,DE0 10 solar,DE0 100 CCGT,DE0 100 CCGT_pos,DE0 100 OCGT,DE0 100 OCGT_pos,DE0 100 onwind,DE0 100 solar,DE0 101 CCGT,DE0 101 CCGT_pos,DE0 101 OCGT,DE0 101 OCGT_pos,DE0 101 onwind,DE0 101 solar,DE0 102 CCGT,DE0 102 CCGT_pos,DE0 102 OCGT,DE0 102 OCGT_pos,DE0 102 onwind,DE0 102 solar,DE0 103 onwind,DE0 103 solar,DE0 104 onwind,DE0 104 solar,DE0 105 onwind,DE0 105 solar,DE0 106 OCGT,DE0 106 OCGT_pos,DE0 106 onwind,DE0 106 solar,DE0 107 onwind,DE0 107 solar,DE0 108 onwind,DE0 108 solar,DE0 109 onwind,DE0 109 solar,DE0 11 OCGT,DE0 11 OCGT_pos,DE0 11 onwind,DE0 11 solar,DE0 110 CCGT,DE0 110 CCGT_pos,DE0 110 OCGT,DE0 110 OCGT_pos,DE0 110 onwind,DE0 110 solar,DE0 111 onwind,DE0 111 solar,DE0 112 OCGT,DE0 112 OCGT_pos,DE0 112 onwind,DE0 112 solar,DE0 113 onwind,DE0 113 solar,DE0 114 onwind,DE0 114 solar,DE0 115 onwind,DE0 115 solar,DE0 117 onwind,DE0 117 solar,DE0 118 onwind,DE0 118 solar,DE0 119 OCGT,DE0 119 OCGT_pos,DE0 119 onwind,DE0 119 solar,DE0 12 CCGT,DE0 12 CCGT_pos,DE0 12 OCGT,DE0 12 OCGT_pos,DE0 12 onwind,DE0 12 solar,DE0 120 OCGT,DE0 120 OCGT_pos,DE0 120 onwind,DE0 120 solar,DE0 121 onwind,DE0 121 solar,DE0 122 OCGT,DE0 122 OCGT_pos,DE0 122 offwind-ac,DE0 122 offwind-dc,DE0 122 onwind,DE0 122 solar,DE0 123 OCGT,DE0 123 OCGT_pos,DE0 123 onwind,DE0 123 solar,DE0 124 onwind,DE0 124 solar,DE0 125 onwind,DE0 125 solar,DE0 126 OCGT,DE0 126 OCGT_pos,DE0 126 onwind,DE0 126 solar,DE0 127 OCGT,DE0 127 OCGT_pos,DE0 127 onwind,DE0 127 solar,DE0 128 onwind,DE0 128 solar,DE0 129 onwind,DE0 129 solar,DE0 13 OCGT,DE0 13 OCGT_pos,DE0 13 onwind,DE0 13 solar,DE0 130 onwind,DE0 130 solar,DE0 131 onwind,DE0 131 solar,DE0 132 OCGT,DE0 132 OCGT_pos,DE0 132 onwind,DE0 132 solar,DE0 133 OCGT,DE0 133 OCGT_pos,DE0 133 onwind,DE0 133 solar,DE0 134 onwind,DE0 134 solar,DE0 135 onwind,DE0 135 solar,DE0 136 onwind,DE0 136 solar,DE0 137 CCGT,DE0 137 CCGT_pos,DE0 137 OCGT,DE0 137 OCGT_pos,DE0 137 onwind,DE0 137 solar,DE0 138 onwind,DE0 138 solar,DE0 139 OCGT,DE0 139 OCGT_pos,DE0 139 onwind,DE0 139 solar,DE0 14 CCGT,DE0 14 CCGT_pos,DE0 14 OCGT,DE0 14 OCGT_pos,DE0 14 onwind,DE0 14 solar,DE0 140 OCGT,DE0 140 OCGT_pos,DE0 140 onwind,DE0 140 solar,DE0 141 OCGT,DE0 141 OCGT_pos,DE0 141 onwind,DE0 141 solar,DE0 142 onwind,DE0 142 solar,DE0 143 onwind,DE0 143 solar,DE0 144 onwind,DE0 144 solar,DE0 145 onwind,DE0 145 solar,DE0 146 onwind,DE0 146 solar,DE0 147 onwind,DE0 147 solar,DE0 148 CCGT,DE0 148 CCGT_pos,DE0 148 OCGT,DE0 148 OCGT_pos,DE0 148 onwind,DE0 148 solar,DE0 149 onwind,DE0 149 solar,DE0 15 CCGT,DE0 15 CCGT_pos,DE0 15 OCGT,DE0 15 OCGT_pos,DE0 15 onwind,DE0 15 solar,DE0 151 offwind-ac,DE0 151 offwind-dc,DE0 151 onwind,DE0 151 solar,DE0 152 onwind,DE0 152 solar,DE0 153 onwind,DE0 153 solar,DE0 154 CCGT,DE0 154 CCGT_pos,DE0 154 OCGT,DE0 154 OCGT_pos,DE0 154 onwind,DE0 154 solar,DE0 155 OCGT,DE0 155 OCGT_pos,DE0 155 onwind,DE0 155 solar,DE0 156 onwind,DE0 156 solar,DE0 158 onwind,DE0 158 solar,DE0 159 CCGT,DE0 159 CCGT_pos,DE0 159 OCGT,DE0 159 OCGT_pos,DE0 159 onwind,DE0 159 solar,DE0 16 CCGT,DE0 16 CCGT_pos,DE0 16 OCGT,DE0 16 OCGT_pos,DE0 16 offwind-ac,DE0 16 onwind,DE0 16 solar,DE0 161 onwind,DE0 161 solar,DE0 162 OCGT,DE0 162 OCGT_pos,DE0 162 onwind,DE0 162 solar,DE0 163 onwind,DE0 163 solar,DE0 164 onwind,DE0 164 solar,DE0 165 OCGT,DE0 165 OCGT_pos,DE0 165 onwind,DE0 165 solar,DE0 166 OCGT,DE0 166 OCGT_pos,DE0 166 onwind,DE0 166 solar,DE0 167 onwind,DE0 167 solar,DE0 168 onwind,DE0 168 solar,DE0 169 CCGT,DE0 169 CCGT_pos,DE0 169 OCGT,DE0 169 OCGT_pos,DE0 169 onwind,DE0 169 solar,DE0 17 CCGT,DE0 17 CCGT_pos,DE0 17 OCGT,DE0 17 OCGT_pos,DE0 17 onwind,DE0 17 solar,DE0 170 CCGT,DE0 170 CCGT_pos,DE0 170 OCGT,DE0 170 OCGT_pos,DE0 170 onwind,DE0 170 solar,DE0 171 OCGT,DE0 171 OCGT_pos,DE0 171 onwind,DE0 171 solar,DE0 173 onwind,DE0 173 solar,DE0 174 onwind,DE0 174 solar,DE0 175 onwind,DE0 175 sola

name
1      3396.205223
2      5094.307835
3         0.000000
4      5094.307835
5      5094.307835
          ...     
285    3396.205223
286    3396.205223
287    1138.340255
288    1698.102612
289    3396.205223
Name: s_nom_opt, Length: 289, dtype: float64

In [79]:
# Extract results from dispatch & redispatch optimization

network = l_networks_bat[1]
network_rd = l_networks[2]
network_rd_bat = l_networks_bat[2]



print(network.objective)
print(network_rd.objective)
print(network_rd_bat.objective)

# cost reduction
delta_objective = network_rd.objective - network_rd_bat.objective
print(round(delta_objective, 2))

# Dispatch time series
df_dispatch_t = network.generators_t.p.copy()
df_dispatch_t["Sum"] = df_dispatch_t.sum(axis = 1)

# Redispatch w/o batteries
# ------------------------

# Redispatch df
df_redispatch_t = network_rd.generators_t.p.copy()
cols_rd = [column for column in list(df_redispatch_t) if "pos" in column or "neg" in column]         
df_redispatch_t = df_redispatch_t.copy()[cols_rd]

# Resulting dispatch after redispatch
#display(df_redispatch_t)

# Absolute redispatch df
df_redispatch_abs = df_redispatch_t.abs()
df_redispatch_abs["Sum"] = df_redispatch_abs.sum(axis = 1)

# df for positive rd
cols_rd_pos = [column for column in list(df_redispatch_t) if "pos" in column]         
df_redispatch_t_pos = df_redispatch_t.copy()[cols_rd_pos]
df_redispatch_t_pos["Sum pos"] = df_redispatch_t_pos.sum(axis = 1)

# df for negative rd
cols_rd_neg = [column for column in list(df_redispatch_t) if "neg" in column]         
df_redispatch_t_neg = df_redispatch_t.copy()[cols_rd_neg]
df_redispatch_t_neg["Sum neg"] = df_redispatch_t_neg.sum(axis = 1)

# Add sums to redispatch df
df_redispatch_t["Sum pos"] = df_redispatch_t_pos["Sum pos"]
df_redispatch_t["Sum neg"] = df_redispatch_t_neg["Sum neg"]


# Redispatch w/ batteries
# ------------------------

# Redispatch time series with batteries
df_redispatch_t_bat = network_rd_bat.generators_t.p.copy()
cols_rd = [column for column in list(df_redispatch_t_bat) if "pos" in column or "neg" in column]         
df_redispatch_t_bat = df_redispatch_t_bat[cols_rd].copy()

# Absolute redispatch df
df_redispatch_abs = df_redispatch_t_bat.abs()
df_redispatch_abs["Sum"] = df_redispatch_abs.sum(axis = 1)

# df for positive rd
cols_rd_pos = [column for column in list(df_redispatch_t_bat) if "pos" in column]         
df_redispatch_t_bat_pos = df_redispatch_t_bat.copy()[cols_rd_pos]
df_redispatch_t_bat_pos["Sum pos"] = df_redispatch_t_bat_pos.sum(axis = 1)

# df for negative rd
cols_rd_neg = [column for column in list(df_redispatch_t_bat) if "neg" in column]         
df_redispatch_t_bat_neg = df_redispatch_t_bat.copy()[cols_rd_neg]
df_redispatch_t_bat_neg["Sum neg"] = df_redispatch_t_bat_neg.sum(axis = 1)

# Add sums to redispatch df
df_redispatch_t_bat["Sum pos"] = df_redispatch_t_bat_pos["Sum pos"]
df_redispatch_t_bat["Sum neg"] = df_redispatch_t_bat_neg["Sum neg"]

# df with difference of redispatch
df_bat_vs_nobat = pd.concat([df_redispatch_t[["Sum pos", "Sum neg"]], df_redispatch_t_bat[["Sum pos", "Sum neg"]]], axis = 1)
df_bat_vs_nobat["Difference positive"] = abs(df_bat_vs_nobat.iloc[:,0] - df_bat_vs_nobat.iloc[:,2])
df_bat_vs_nobat["Difference negative"] = - abs(df_bat_vs_nobat.iloc[:,1] - df_bat_vs_nobat.iloc[:,3])
#display(df_bat_vs_nobat)

# battery charging operations (of one in the north and one in the south)
df_bess = network_rd_bat.stores
df_bess_t_p = network_rd_bat.stores_t.p

display(network_rd_bat.stores_t.p)
display(network_rd_bat.stores_t.e)
display(network_rd_bat.stores.e_nom.T)
df_bess_SOC = network_rd_bat.stores_t.e / network_rd_bat.stores.e_nom.T
print("df_bess_SOC")
display(df_bess_SOC)


# ------------------------
# Plots w/o bat
# ------------------------

# # Dispatch time series
# cols = list(df_dispatch_t)[:-1]
# fig, ax  = plt.subplots(figsize = (10, 5))
# ax.set_ylabel('MW Redispatch',  fontsize = 12)
# ax = df_dispatch_t.loc[:, cols].plot.bar(stacked = True, ax = ax, title = "Dispatch after market clearing", legend = True, fontsize = 12)
# ax.set_facecolor('white')
# ax.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# # Redispatch time series (pos vs. neg aggregate) 
# cols_0 = list(df_redispatch_t)[-2:]
# fig, ax_0  = plt.subplots(figsize = (10, 5))
# ax_0.set_ylabel('MW Redispatch',  fontsize = 12)
# ax_0 = df_redispatch_t.loc[:, cols_0].plot.bar(stacked = True, ax = ax_0, title = "Amount of redispatch", legend = True, fontsize = 12)
# ax_0.set_facecolor('white')
# ax_0.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# # Redispatch time series (aggregate) 
# cols_1 = list(df_redispatch_abs)[:-1]
# fig, ax_1  = plt.subplots(figsize = (10, 5))
# ax_1.set_ylabel('MW Redispatch',  fontsize = 12)
# ax_1 = df_redispatch_abs.loc[:, cols_1].plot.bar(stacked = True, ax = ax_1, title = "Aggregated amount of redispatch", legend = True, fontsize = 12)
# ax_1.set_facecolor('white')
# ax_1.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# # Redispatch time series (positive rd) 
# cols_2 = list(df_redispatch_t_pos)[:-1]
# fig, ax_2  = plt.subplots(figsize = (10, 5))
# ax_2.set_ylabel('MW Redispatch',  fontsize = 12)
# ax_2 = df_redispatch_t_pos.loc[:, cols_2].plot.bar(stacked = True, ax = ax_2, title = "Amount of positive redispatch (ramp up)", legend = True, fontsize = 12)
# ax_2.set_facecolor('white')
# ax_2.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# # Redispatch time series (negative rd) 
# cols_3 = list(df_redispatch_t_neg)[:-1]
# fig, ax_3  = plt.subplots(figsize = (10, 5))
# ax_3.set_ylabel('MW Redispatch',  fontsize = 12)
# ax_3 = df_redispatch_t_neg.loc[:, cols_3].plot.bar(stacked = True, ax = ax_3, title = "Amount of negative redispatch (curtailment)", legend = True, fontsize = 12)
# ax_3.set_facecolor('white')
# ax_3.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# # Plots w/ bat
# # ------------------------

# Redispatch time series (pos vs. neg aggregate) 
cols_4 = list(df_redispatch_t_bat)[-2:]
fig, ax_4  = plt.subplots(figsize = (10, 5))
ax_4.set_ylabel('MW Redispatch',  fontsize = 12)
ax_4 = df_redispatch_t_bat.loc[:, cols_4].plot.bar(stacked = True, ax = ax_4, title = "Amount of positive and negative redispatch", legend = True, fontsize = 12)
ax_4.set_facecolor('white')
ax_4.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# Difference positive vs negative redispatch 
cols_5 = list(df_bat_vs_nobat)[-2:]
fig, ax_5  = plt.subplots(figsize = (10, 5))
ax_5.set_ylabel('MW Redispatch',  fontsize = 12)
ax_5 = df_bat_vs_nobat.loc[:, cols_5].plot.bar(stacked = True, ax = ax_5, title = "Difference between redispatch with and without bat", legend = True, fontsize = 12)
ax_5.set_facecolor('white')
ax_5.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# Plots battery insights
# ------------------------

# Battery charging operations (battery in the north)
cols_6 = ["BESS_1","BESS_2","BESS_3"]
fig, ax_6  = plt.subplots(figsize = (10, 5))
ax_6.set_ylabel('MW Redispatch',  fontsize = 12)
ax_6 = df_bess_t_p.loc[:, cols_6].plot.bar(stacked = True, ax = ax_6, title = "Battery active power in redispatch", legend = True, fontsize = 12)
ax_6.set_facecolor('white')
ax_6.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)

# Battery SOC
cols_7 = ["BESS_1","BESS_2","BESS_3"]
fig, ax_7  = plt.subplots(figsize = (10, 5))
ax_7.set_ylabel('Battery SOC',  fontsize = 12)
ax_7 = df_bess_SOC.loc[:, cols_7].plot.line(stacked = False, ax = ax_7, title = "Battery SOC during redispatch", legend = True, fontsize = 12)
ax_7.set_facecolor('white')
ax_7.grid(color = 'grey', linestyle = '-.', linewidth = 1.0)



662556.3305907235


AttributeError: 'Network' object has no attribute 'objective'